# Starter Notebook: DES Encryption with CBC Mode

**FOR TA CORRECTING: install bitarray library on your ide so the code runs!!!**

In [1]:
from fontTools.misc.textTools import tobytes
from ruamel.yaml import round_trip_load

# DES Encryption with CBC Mode
# Group Members: Youssef Ahmed Elraghy / 13001467 / P02
#                Abdelrahman Mostafa   / 13002064 / P02

# DES Tables
IP = [58, 50, 42, 34, 26, 18, 10, 2, 60, 52, 44, 36, 28, 20, 12, 4,
      62, 54, 46, 38, 30, 22, 14, 6, 64, 56, 48, 40, 32, 24, 16, 8,
      57, 49, 41, 33, 25, 17, 9, 1, 59, 51, 43, 35, 27, 19, 11, 3,
      61, 53, 45, 37, 29, 21, 13, 5, 63, 55, 47, 39, 31, 23, 15, 7]

FP = [40, 8, 48, 16, 56, 24, 64, 32, 39, 7, 47, 15, 55, 23, 63, 31,
      38, 6, 46, 14, 54, 22, 62, 30, 37, 5, 45, 13, 53, 21, 61, 29,
      36, 4, 44, 12, 52, 20, 60, 28, 35, 3, 43, 11, 51, 19, 59, 27,
      34, 2, 42, 10, 50, 18, 58, 26, 33, 1, 41, 9, 49, 17, 57, 25]

E = [32, 1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 8, 9, 10, 11, 12, 13, 12, 13, 14, 15, 16, 17,
     16, 17, 18, 19, 20, 21, 20, 21, 22, 23, 24, 25, 24, 25, 26, 27, 28, 29, 28, 29, 30, 31, 32, 1]

P = [16, 7, 20, 21, 29, 12, 28, 17, 1, 15, 23, 26, 5, 18, 31, 10,
     2, 8, 24, 14, 32, 27, 3, 9, 19, 13, 30, 6, 22, 11, 4, 25]

S_BOXES = [
    [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
     [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
     [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
     [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],

    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
     [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
     [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
     [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],

    [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
     [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
     [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
     [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
     [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
     [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
     [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
     [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
     [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
     [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],

    [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
     [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
     [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
     [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
     [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
     [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
     [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
     [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
     [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
     [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]
]

# 16 round keys (48 bits each)
round_keys = ["010101010101010101010101010101010101010101010101" for _ in range(16)]

In [2]:
from bitarray import bitarray

# key rotations for proper results from feistel in DES only
def rotate_left(bits, n):
    return bits[n:] + bits[:n]

round_keys = [bitarray(key) for key in round_keys]
# call shift method on all round keys
for i in range(len(round_keys)):
    round_keys[i] = rotate_left(round_keys[i], i)
round_keys = [key.to01() for key in round_keys] # string to bits conv


# XOR method for xoring step of DES
def xor(b1, b2):
    return bitarray([b1[i] != b2[i] for i in range(len(b1))])

# text to bits conversion method (includes padding)
def t2b(text, size=64):
    bits = bitarray()
    bits.frombytes(text.encode('utf-8'))
    while len(bits) < size:
        bits.extend('0')
    return bits[:size]

# bits to text conversion method
def b2t(bits):
    return bits.tobytes().decode('utf-8', 'ignore')

# Permutation method -> swaps bits according to ptable
def permute(block, Ptable):
    return bitarray([block[i-1] for i in Ptable])

# s-box substitutions
def s_box(bits):
    output = bitarray()
    for i in range(8):
        chunk = bits[i*6:(i+1)*6]
        row = int(str(chunk[0]) + str(chunk[5]), 2)
        col = int(chunk[1:5].to01(), 2)
        output.extend(f"{S_BOXES[i][row][col]:04b}")
    return output

# feistel function method
def feistel(right, key):
    expand = permute(right, E)
    print(f"Expanded right: {expand}")
    xord = xor(expand, key)
    print(f"XOR with key: {xord}")
    sub = s_box(xord)
    print(f"S-box output: {sub}")
    return permute(sub, P)

# encryption function (combines all the helper/step methods we defined above)
def encrypt(pt, keys):
    bin_text = t2b(pt) # converts our plaintext into binary
    print(f"Binary text: {bin_text}")
    permuted = permute(bin_text, IP) # performs initial permutation on binary form of plaintext
    print(f"Initial permutation: {permuted}")
    left, right = permuted[:32], permuted[32:] # splits block into left and right segments
    print(f"Initial left: {left}, Initial right: {right}")
    # 16 rounds of feistel for 16 keys
    for i, key in enumerate(keys):
        new_r = xor(left, feistel(right, bitarray(key))) # gets new right by xoring the left block with the output of the feistel of the current right and key
        left, right = right, new_r # shifts so the left becomes the right and the right becomes the new right
        print(f"\nRound {i+1} - Left: {left}, Right: {right}")
    combined = left + right # combines the final blocks
    print(f"Combined before final permutation: {combined}")
    return permute(combined, FP) # performs permutation on the combined final blocks using the final permutation table (given)

# CBC mode ecrypt
def encrypt_cbc(pt, keys, iv):
    pt_blocks = [pt[i:i+8] for i in range(0, len(pt), 8)]
    iv_bits = t2b(iv)
    ciphertext = bitarray()
    for block in pt_blocks:
        block_bits = t2b(block) # switch block from text to binary
        block_bits = xor(block_bits, iv_bits) # xor block with the IV/prev_CT (CBC steps)
        ct_bits = encrypt(b2t(block_bits), keys) # encrypt the xored block
        ciphertext.extend(ct_bits) # add encrypted block to final ciphertext
        iv_bits = ct_bits # next iteration of IV is the current ciphertext(block)
    return ciphertext # returns ciphertext bits

**DES Encryption in CBC mode (>64-bit blocks + IVs)**

In [3]:
from bitarray import bitarray
iv = "12345678"  # Initialization vector
pt = "desHello more than 8 bytes"
ct = encrypt_cbc(pt, round_keys, iv)
print('\nCiphertext after DES encryption with CBC mode (binary):', ct)
hexct = ct.tobytes().hex()
print('Ciphertext after DES encryption with CBC mode (hex):', hexct)
ascii_ct = (bytes.fromhex(hexct)).decode('utf-8', 'ignore')
print('Ciphertext after DES encryption with CBC mode (ascii):', ascii_ct)
# encrypted text will only differ if the IV changes

Binary text: bitarray('0101010101010111010000000111110001010000010110100101101101010111')
Initial permutation: bitarray('1111111111111011100010111100001100000000000010000110100011100010')
Initial left: bitarray('11111111111110111000101111000011'), Initial right: bitarray('00000000000010000110100011100010')
Expanded right: bitarray('000000000000000001010000001101010001011100000100')
XOR with key: bitarray('010101010101010100000101011000000100001001010001')
S-box output: bitarray('11000001110010111101101001001100')

Round 1 - Left: bitarray('00000000000010000110100011100010'), Right: bitarray('01000110000011100100101010100011')
Expanded right: bitarray('101000001100000001011100001001010101010100000110')
XOR with key: bitarray('000010100110101011110110100011111111111110101100')
S-box output: bitarray('01001011100111101000110100101110')

Round 2 - Left: bitarray('01000110000011100100101010100011'), Right: bitarray('01011101010000101001110110001010')
Expanded right: bitarray('00101111101010

**DES only encryption for 64-bit block**

In [4]:
from bitarray import bitarray
pt = "DESHello"
ct = encrypt(pt, round_keys)
print('\nCiphertext after DES only (binary): ', ct)
hexct = ct.tobytes().hex()
print('\nCiphertext after DES only (hex): ', hexct)
ascii_ct = (bytes.fromhex(hexct)).decode('utf-8', 'ignore')
print('\nCiphertext after DES only (ascii): ', ascii_ct)

Binary text: bitarray('0100010001000101010100110100100001100101011011000110110001101111')
Initial permutation: bitarray('1111111100000100111100111001011000000000111100001110100010000100')
Initial left: bitarray('11111111000001001111001110010110'), Initial right: bitarray('00000000111100001110100010000100')
Expanded right: bitarray('000000000001011110100001011101010001010000001000')
XOR with key: bitarray('010101010100001011110100001000000100000101011101')
S-box output: bitarray('11000010010000110111101010111001')

Round 1 - Left: bitarray('00000000111100001110100010000100'), Right: bitarray('00000101111000010111111100010111')
Expanded right: bitarray('100000001011111100000010101111111110100010101110')
XOR with key: bitarray('001010100001010110101000000101010100001000000100')
S-box output: bitarray('11111101011111000010001111111000')

Round 2 - Left: bitarray('00000101111000010111111100010111'), Right: bitarray('00001110010010010001111001010011')
Expanded right: bitarray('10000101110000